In [1]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, confusion_matrix
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')

/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #620
  (fname, cnt))


In [2]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:


import numpy as np
import numbers
import os
import pandas as pd
import random
import time
import warnings

from abc import ABCMeta
from itertools import product
from scipy.stats import ttest_ind
from sklearn.base import BaseEstimator, ClassifierMixin, MetaEstimatorMixin, clone, is_classifier
from sklearn.exceptions import FitFailedWarning
from sklearn.externals import joblib, six
from sklearn.feature_selection.base import SelectorMixin
from sklearn.feature_selection.univariate_selection import _BaseFilter, _clean_nans
from sklearn.metrics.scorer import _check_multimetric_scoring, check_scoring
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection._validation import _index_param_value, _score
from sklearn.model_selection._split import check_cv
from sklearn.utils import as_float_array, check_X_y, safe_sqr 
from sklearn.utils._joblib import Parallel, delayed, logger
from sklearn.utils.metaestimators import _safe_split
from sklearn.utils.validation import indexable, check_is_fitted, _num_samples
from traceback import format_exception_only
#from utils import SafeEval


VERSION = '0.1.1'

memory = joblib.Memory('./memory_cache')
N_JOBS = int(os.environ.get('GALAXY_SLOTS', 1))


class MemoryFit(object):
    def fit(self, *args, **kwargs):
        fit = memory.cache(super(MemoryFit, self).fit)
        cached_self = fit(*args, **kwargs)
        vars(self).update(vars(cached_self))


class IRAPSCore(object):
    def __init__(self, n_iter=1000, responsive_thres=-1,
                resistant_thres=0, verbose=0, random_state=None):
        self.n_iter = n_iter
        self.responsive_thres = responsive_thres
        self.resistant_thres = resistant_thres
        self.verbose = verbose
        self.random_state = random_state

    def fit(self, X, y):
        """
        X: array-like (n_samples x n_features)
        y: 1-d array-like (n_samples)
        """
        X, y = check_X_y(X, y, ['csr', 'csc'], multi_output=True)
        #each iteration select a random number of random subset of training samples
        # this is somewhat different from the original IRAPS method, but effect is almost the same.
        SAMPLE_SIZE = [0.25, 0.75]
        n_samples = X.shape[0]
        pvalues = None
        fold_changes = None
        base_values = None

        i = 0
        while i < self.n_iter:
            ## TODO: support more random_state/seed
            if self.random_state is None:
                n_select = random.randint(int(n_samples*SAMPLE_SIZE[0]), int(n_samples*SAMPLE_SIZE[1]))
                index = random.sample(list(range(n_samples)), n_select)
            else:
                n_select = random.Random(i).randint(int(n_samples*SAMPLE_SIZE[0]), int(n_samples*SAMPLE_SIZE[1]))
                index = random.Random(i).sample(list(range(n_samples)), n_select)
            X_selected, y_selected = X[index], y[index]
            
            # Spliting by z_scores.
            y_selected = (y_selected - y_selected.mean())/y_selected.std()
            X_selected_responsive = X_selected[y_selected <= self.responsive_thres]
            X_selected_resistant = X_selected[y_selected > self.resistant_thres]

            # For every iteration, at least 5 responders are selected
            if X_selected_responsive.shape[0] < 5:
                continue

            if self.verbose:
                print("Working on iteration %d/%d, %s/%d samples were responder/selected."\
                        %(i+1, self.n_iter, X_selected_responsive.shape[0], n_select))
            i += 1

            # p_values
            _, p = ttest_ind(X_selected_responsive, X_selected_resistant, axis=0, equal_var=False)
            if pvalues is None:
                pvalues = p
            else:
                pvalues = np.vstack((pvalues, p))

            # fold_change == mean change?
            # TODO implement other normalization method
            responsive_mean = X_selected_responsive.mean(axis=0)
            resistant_mean = X_selected_resistant.mean(axis=0)
            mean_change = responsive_mean - resistant_mean
            #mean_change = np.select([responsive_mean >= resistant_mean, responsive_mean < resistant_mean],
            #                        [responsive_mean / resistant_mean, -resistant_mean / responsive_mean])
            # mean_change could be adjusted by power of 2
            # mean_change = 2**mean_change if mean_change>0 else -2**abs(mean_change)
            if fold_changes is None:
                fold_changes = mean_change
            else:
                fold_changes = np.vstack((fold_changes, mean_change))

            if base_values is None:
                base_values = resistant_mean
            else:
                base_values = np.vstack((base_values, resistant_mean))

        self.fold_changes_ = np.asarray(fold_changes)
        self.pvalues_ = np.asarray(pvalues)
        self.base_values_ = np.asarray(base_values)

        return self


class CachedIRAPSCore(MemoryFit, IRAPSCore):
    pass


class IRAPSClassifier(six.with_metaclass(ABCMeta, _BaseFilter, BaseEstimator, ClassifierMixin)):
    """
    Extend the bases of both sklearn feature_selector and classifier.
    From sklearn base:
        get_params()
        set_params()
    From sklearn feature_selector:
        get_support()
        fit_transform(X)
        transform(X)
    From sklearn classifier:
        predict(X)
        predict_proba(X)
    New:
        get_signature()
    Properties:
        discretize_value

    """
    def __init__(self, iraps_core, p_thres=1e-4, fc_thres=0.1, occurance=0.8, discretize='z_score'):
        self.iraps_core = iraps_core
        self.p_thres = p_thres
        self.fc_thres = fc_thres
        self.occurance = occurance
        self.discretize = discretize

    def fit(self, X, y):
        # allow pre-fitted iraps_core here
        if not hasattr(self.iraps_core, 'pvalues_'):
            self.iraps_core.fit(X, y)

        pvalues = as_float_array(self.iraps_core.pvalues_, copy=True)
        ## why np.nan is here?
        pvalues[np.isnan(pvalues)] = np.finfo(pvalues.dtype).max

        fold_changes = as_float_array(self.iraps_core.fold_changes_, copy=True)
        fold_changes[np.isnan(fold_changes)] = 0.0

        base_values = as_float_array(self.iraps_core.base_values_, copy=True)

        p_thres = self.p_thres
        fc_thres = self.fc_thres
        occurance = self.occurance

        mask_0 = np.zeros(pvalues.shape, dtype=np.int32)
        # mark p_values less than the threashold
        mask_0[pvalues <= p_thres] = 1
        # mark fold_changes only when greater than the threashold
        mask_0[abs(fold_changes) < fc_thres] = 0

        # count the occurance and mask greater than the threshold
        counts = mask_0.sum(axis=0)
        occurance_thres = int(occurance * self.iraps_core.n_iter)
        mask = np.zeros(counts.shape, dtype=bool)
        mask[counts >= occurance_thres] = 1

        # generate signature
        fold_changes[mask_0 == 0] = 0.0
        signature = fold_changes[:, mask].sum(axis=0) / counts[mask]
        signature = np.vstack((signature, base_values[:, mask].mean(axis=0)))

        self.signature_ = np.asarray(signature)
        self.mask_ = mask
        self.discretize_value = y.mean() - y.std()

        return self


    def _get_support_mask(self):
        """
        return mask of feature selection indices
        """
        check_is_fitted(self, 'mask_')

        return self.mask_

    def get_signature(self, min_size=1):
        """
        return signature
        """
        #TODO: implement minimum size of signature
        # It's not clearn whether min_size could impact prediction performance
        check_is_fitted(self, 'signature_')

        if self.signature_.shape[1] >= min_size:
            return self.signature_
        else:
            return None

    def predict_proba(self, X):
        """
        compute the correlation coefficient with irpas signature
        """
        signature = self.get_signature()
        if signature is None:
            print('The classifier got None signature or the number of sinature feature is less than minimum!')
            return

        X_transformed = self.transform(X) - signature[1]
        corrcoef = np.array([np.corrcoef(signature[0], e)[0][1] for e in X_transformed])
        corrcoef[np.isnan(corrcoef)] = np.finfo(np.float32).min

        return corrcoef

    def predict(self, X, clf_threshold=0.4):
        return self.predict_proba(X) > clf_threshold

In [4]:
rna = pd.read_csv('./drug_respond/(5Z)-7-Oxozeaenol.csv', sep='\t')
rna

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,RP11-309M23.1 (ENSGR0000237531),AMDP1 (ENSGR0000237801),BX649553.1 (ENSGR0000263835),BX649553.2 (ENSGR0000263980),BX649553.3 (ENSGR0000264510),BX649553.4 (ENSGR0000264819),RN7SL355P (ENSGR0000265350),MIR3690 (ENSGR0000265658),AL732314.1 (ENSGR0000266731),(5Z)-7-Oxozeaenol
0,2.650765,0.000000,6.216843,3.427606,4.672991,0.014355,0.111031,5.803744,6.900867,5.287251,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.862564
1,3.001802,0.000000,6.781229,4.150560,3.839960,0.000000,0.298658,7.425510,6.554589,4.295723,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.759749
2,4.590362,0.000000,6.647890,1.899176,3.360364,0.028569,0.536053,5.762615,3.226509,4.024142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.658579
3,5.881175,0.000000,6.643135,2.039138,5.027243,0.189034,1.575312,6.189232,3.427606,3.980939,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.821017
4,5.045268,0.000000,6.988571,1.883621,3.517276,0.000000,3.795975,6.383704,4.651339,5.055282,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.710453
5,5.943218,0.000000,6.501280,1.937344,4.078951,0.000000,0.432959,5.942280,4.527946,4.472488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.662955
6,0.150560,0.000000,5.460415,2.375735,4.360364,0.042644,0.298658,0.356144,4.182692,5.354382,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.578093
7,3.438293,0.000000,6.313790,2.286881,4.152995,0.000000,1.691534,6.444270,4.759156,4.106013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.662305
8,2.980025,0.000000,7.118630,1.903038,4.533563,0.000000,3.090853,6.827565,3.969933,3.872829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.699442
9,4.316146,0.000000,6.903400,1.871844,3.569248,0.000000,3.912650,4.234961,3.519793,4.698218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.574803


In [5]:
X, y = rna.iloc[:, :-1], rna.iloc[:, -1]
y

0      0.862564
1      0.759749
2      0.658579
3      0.821017
4      0.710453
5      0.662955
6      0.578093
7      0.662305
8      0.699442
9      0.574803
10     0.634965
11     0.664182
12     0.406618
13     0.586659
14     0.413623
15     0.677848
16     0.647090
17     0.042457
18     0.625574
19     0.738274
20     0.469464
21     0.765864
22     0.534056
23     0.683547
24     0.721389
25     0.693708
26     0.670416
27     0.629251
28     0.768977
29     0.694602
         ...   
571    0.575736
572    0.984579
573    0.429754
574    0.769001
575    0.921043
576    0.903164
577    0.543535
578    0.576542
579    0.838408
580    0.805498
581    0.439971
582    0.685980
583    0.698297
584    0.739760
585    0.550116
586    0.635176
587    0.251275
588    0.168720
589    0.732249
590    0.610444
591    0.908638
592    0.606007
593    0.652107
594    0.489042
595    0.480574
596    0.977626
597    0.882111
598    0.439166
599    0.650792
600    0.899007
Name: (5Z)-7-Oxozeaenol,

In [6]:
cv = KFold(10, shuffle=True, random_state=0)
scores = []
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X.values[train_index], y[train_index]
    X_test, y_test = X.values[test_index], y[test_index]
    iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=None)
    iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
    iraps.fit(X_train, y_train)
    y_p = iraps.predict_proba(X_test)
    y_t = y_test < iraps.discretize_value
    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    scores.append(auc)
print("Mean: %f" %(sum(scores)/float(len(scores))))

AUC: 0.877358
AUC: 0.858173
AUC: 0.795918
AUC: 0.710000
AUC: 0.857143
AUC: 0.800000
AUC: 0.631751
AUC: 0.818000
AUC: 0.769063
AUC: 0.609164
Mean: 0.772657


## new drug  Olaparib_(1)

In [7]:
rna = pd.read_csv('./drug_respond/Olaparib_(1).tsv', sep='\t')
rna

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,RP4-671G15.2 (ENSG00000273483),OR6R2P (ENSG00000273484),RP11-225H22.7 (ENSG00000273485),RP11-731C17.2 (ENSG00000273486),RP4-621B10.8 (ENSG00000273487),RP11-114I8.4 (ENSG00000273488),RP11-180C16.1 (ENSG00000273489),AP000230.1 (ENSG00000273492),RP11-80H18.4 (ENSG00000273493),Olaparib (1)
0,2.650765,0.000000,6.216843,3.427606,4.672991,0.014355,0.111031,5.803744,6.900867,5.287251,...,0.910733,0.0,0.443607,1.555816,0.250962,2.432959,3.174726,0.014355,0.000000,0.978863
1,3.001802,0.000000,6.781229,4.150560,3.839960,0.000000,0.298658,7.425510,6.554589,4.295723,...,0.356144,0.0,0.000000,0.748461,0.070389,1.570463,3.065228,0.111031,0.000000,0.985041
2,4.590362,0.000000,6.647890,1.899176,3.360364,0.028569,0.536053,5.762615,3.226509,4.024142,...,0.150560,0.0,0.565597,0.432959,0.000000,0.815575,1.454176,0.014355,0.000000,0.922960
3,5.881175,0.000000,6.643135,2.039138,5.027243,0.189034,1.575312,6.189232,3.427606,3.980939,...,0.000000,0.0,0.000000,0.176323,0.097611,0.475085,2.049631,0.084064,0.000000,0.915580
4,5.045268,0.000000,6.988571,1.883621,3.517276,0.000000,3.795975,6.383704,4.651339,5.055282,...,0.124328,0.0,0.263034,0.584963,0.000000,1.570463,1.970854,0.150560,0.000000,0.938053
5,5.943218,0.000000,6.501280,1.937344,4.078951,0.000000,0.432959,5.942280,4.527946,4.472488,...,0.124328,0.0,0.000000,0.344828,0.028569,0.411426,1.244887,0.000000,0.000000,0.978103
6,0.150560,0.000000,5.460415,2.375735,4.360364,0.042644,0.298658,0.356144,4.182692,5.354382,...,0.000000,0.0,0.214125,0.963474,0.000000,1.636915,1.978196,0.084064,1.495695,0.824990
7,4.316146,0.000000,6.903400,1.871844,3.569248,0.000000,3.912650,4.234961,3.519793,4.698218,...,0.275007,0.0,0.176323,0.695994,0.000000,0.799087,1.655352,0.000000,0.000000,0.973685
8,4.724650,0.000000,6.957450,1.835924,3.682573,0.000000,0.454176,5.825277,3.918386,4.197708,...,0.238787,0.0,0.879706,0.887525,0.000000,1.070389,1.495695,0.000000,0.000000,0.925170
9,5.108106,0.000000,6.888500,2.207893,4.008092,0.505891,2.611172,6.316146,5.361417,4.190615,...,0.495695,0.0,0.641546,1.344828,0.189034,1.839960,1.887525,0.163499,0.000000,0.901699


In [8]:
X, y = rna.iloc[:, :-1], rna.iloc[:, -1]
cv = KFold(10, shuffle=True, random_state=0)
scores = []
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X.values[train_index], y[train_index]
    X_test, y_test = X.values[test_index], y[test_index]
    iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=None)
    iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
    iraps.fit(X_train, y_train)
    y_p = iraps.predict_proba(X_test)
    y_t = y_test < iraps.discretize_value
    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    scores.append(auc)
print("Mean: %f" %(sum(scores)/float(len(scores))))

AUC: 0.705729
AUC: 0.830189
AUC: 0.730392
AUC: 0.744898
AUC: 0.897436
AUC: 0.382353
AUC: 0.724490
AUC: 0.610119
AUC: 0.660000
AUC: 0.712000
Mean: 0.699761
